# SoccerNet - Deep Hough Transform - testing with self-trained model

Instructions: DO NOT RUN ALL AUTOMATICALLY. Kindly run the cells for the first and second section manually, and then RUN AFTER ALL CELLS AUTOMATICALLY

## Get Deep Hough Transform repository and adapt (Manual)

In [ ]:
!git clone https://github.com/Hanqer/deep-hough-transform.git

In [ ]:
# fix deep-hough-transform/config.yml file
# TEST_DIR: "data/soccernet_data"

In [ ]:
# fix deep-hough-transform/forward.py file
# Line: 34
# CONFIGS = yaml.load(open(args.config), Loader=yaml.Loader)

## Upload self-trained model to runtime (Manual)

In [ ]:
# upload trained model

## Get SoccerNet data and adapt

In [ ]:
!pip install SoccerNet

In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="SoccerNet/")
mySoccerNetDownloader.downloadDataTask(task="calibration", split=["train","valid","test","challenge"])

In [ ]:
# Prepare Soccernet data
%cd SoccerNet/calibration/

In [ ]:
!unzip train.zip

In [ ]:
%cd train/

In [ ]:
import math 
for x in range(14513):
  lineCount = 0
  x_coords = []
  y_coords = []
  foundCircle = False
  if x >= 10000:
    prefix = ""
  if x < 10000:
    prefix = "0"
  if x < 1000:
    prefix = "00"
  if x < 100:
    prefix = "000"
  if x < 10:
    prefix = "0000"
  with open(prefix + str(x) + '.txt', 'w') as fw:
    with open(prefix + str(x) + '.json', 'r') as fr:
      for line in fr.readlines():
        if 'Circle' not in line and '[' in line:
          foundCircle = False
        if foundCircle == True:
          continue
        if 'Circle' in line:
          foundCircle = True
          continue
        if '"x":' in line:
          x_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
        elif '"y":' in line:
          y_coords.append(int(math.floor(float(line.split(': ')[1].split(',')[0]) * 960)))
      lineCount = len(x_coords) // 2
      fw.write(str(lineCount))
      for i in range(len(x_coords)):
        fw.write(' ' + str(x_coords[i]) + ' ' + str(y_coords[i]))
      fr.close()
    fw.close()

In [ ]:
%mkdir soccernet_data

In [ ]:
%mv *.jpg soccernet_data/

In [ ]:
%mv *.txt soccernet_data/

In [ ]:
%cp -r soccernet_data ../../../deep-hough-transform/data/

## Install Deep Hough Transform dependency

In [ ]:
%cd ../../../deep-hough-transform/

In [ ]:
%cd model/_cdht

In [ ]:
!python setup.py install --user

In [ ]:
%cd ../..

## Prepare testing file for model

In [ ]:
%cd data/training/

In [ ]:
with open('test_SoccerNet.txt', 'w') as f:
  for i in range(0, 14513):
    if i >= 10000:
      prefix = ""
    if i < 10000:
      prefix = "0"
    if i < 1000:
      prefix = "00"
    if i < 100:
      prefix = "000"
    if i < 10:
      prefix = "0000"
    f.write(prefix + str(i) + '\n')

In [ ]:
%cd ../../

## Test the model and save the results (Download from runtime)

In [ ]:
!python forward.py --model '''../{uploaded_model_file_name}''' --tmp results

In [ ]:
!zip -r results.zip results

In [ ]:
# download the results.zip file, which contains the visualizations on the tested data

## Get testing statistics

In [ ]:
!pip install POT

In [ ]:
!pip install Ninja

In [ ]:
#!python --version
#!sudo apt-get update -y
#!sudo apt-get install python3.9
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2
#!python --version

In [ ]:
!python test_nkl.py --pred results/visualize_test/ --gt data/soccernet_data/